# Simple MNist like model for detection

Using as example:
https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from os import listdir
from os.path import isfile, join

import gzip
import pickle
import gc
from random import shuffle

import numpy as np

PATH_TO_INTERMEDIATE = "../data/intermediate"

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12
# input image dimensions
img_rows, img_cols = 28, 28
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("x_train:", x_train.shape)
print(x_train[:1])
print("y_train:", y_train.shape)
print(y_train[:1])

In [ ]:
onlyfiles = [join(PATH_TO_INTERMEDIATE, x) for x in listdir(PATH_TO_INTERMEDIATE) if isfile(join(PATH_TO_INTERMEDIATE, x)) and x.endswith(".pickle")]
shuffle(onlyfiles)

print("Original number of files:", len(onlyfiles))

onlyfiles = onlyfiles[:20]

train_files_size = int(len(onlyfiles) * 0.5)
train_files = onlyfiles[0:train_files_size]
onlyfiles = onlyfiles[train_files_size:]

validation_files_size = int(len(onlyfiles) * 0.5)
validation_files = onlyfiles[0:validation_files_size]
onlyfiles = onlyfiles[validation_files_size:]

test_files_size = len(onlyfiles) 
test_files = onlyfiles[0:test_files_size]
onlyfiles = onlyfiles[test_files_size:]

print('train_files:', len(train_files))
print('validation_files:', len(validation_files))
print('test_files:', len(test_files))


In [ ]:
train_x = []
train_y = []

validation_x = []
validation_y = []

test_x = []
test_y = []
for current_file in train_files:
    sample = []
    with gzip.open(current_file,'r') as f:
        sample = pickle.load(f)
    for current_sample in sample['slices']:
        train_x.append(current_sample['slice'])
        train_y.append(current_sample['is_nuclei'])
train_x = np.array(train_x)
train_y = np.array(train_y)
gc.collect()

In [ ]:
print('train_x:', train_x.shape)
print('train_y:', train_y.shape)

In [ ]:
print(sample.keys())

In [ ]:
print(sample['slices'][0]['slice'].shape)

print(sample['slices'][0])

In [ ]:
total = []

total.append(sample['slices'][0]['slice'])
total.append(sample['slices'][1]['slice'])

total = np.array(total)

print(total.shape)